In [1]:

import itertools
import time
import numpy
import warnings
from io import BytesIO
from http.client import HTTPConnection

from sklearn.metrics.pairwise import pairwise_distances
from sklearn.model_selection import KFold, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import minmax_scale
from sklearn.svm import SVC

# Nous ne voulons pas avoir ce type d'avertissement, qui
# n'est pas utile dans le cadre de ce devoir
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [2]:
def fetchPendigits():
    """
    Cette fonction télécharge le jeu de données pendigits et le
    retourne sous forme de deux tableaux numpy. Le premier élément
    retourné par cette fonction est un tableau de 10992x16 qui
    contient les samples; le second élément est un vecteur de 10992
    qui contient les valeurs cible (target).
    """
    host = 'vision.gel.ulaval.ca'
    url = '/~cgagne/enseignement/apprentissage/A2018/travaux/ucipendigits.npy'
    connection = HTTPConnection(host, port=80, timeout=10)
    connection.request('GET', url)

    rep = connection.getresponse()
    if rep.status != 200:
        print("ERREUR : impossible de télécharger le jeu de données UCI Pendigits! Code d'erreur {}".format(rep.status))
        print("Vérifiez que votre ordinateur est bien connecté à  Internet.")
        return
    stream = BytesIO(rep.read())
    dataPendigits = numpy.load(stream)
    return dataPendigits[:, :-1].astype('float32'), dataPendigits[:, -1]


In [3]:
X,y = fetchPendigits()
X = minmax_scale(X)
len(X[0])

16

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1-(5000/len(X)), random_state=666, shuffle=True)

In [5]:
_times = []
_times.append(time.time())

listK = list(range(1,10))
listKnn = list(map(lambda k:KNeighborsClassifier(n_neighbors=k, weights='distance'),listK))
ErrorKnn = {listKnn[i].n_neighbors:0 for i in range(len(listKnn))}
print("Nombre de classifieurs : ",len(listKnn))

argMinKnn = 0 
Nb_boucle = 30
for _ in range(Nb_boucle):
    kf = KFold(n_splits=3, shuffle=True)
    for train_index, test_index in kf.split(X_train):
        X_train_train = X_train[train_index]
        X_train_test = X_train[test_index]
        y_train_train = y_train[train_index]
        y_train_test = y_train[test_index]
        for i in range(len(listKnn)):
            listKnn[i].fit(X_train_train, y_train_train)
            ErrorKnn[listKnn[i].n_neighbors] += 1 - listKnn[i].score(X_train_test, y_train_test)
    ErrorKnn = {k: v / kf.get_n_splits(X_train) for k, v in ErrorKnn.items()}
    minErrorKnn = min([ErrorKnn[i] for i in ErrorKnn.keys()])
    argmin = list(ErrorKnn.keys())[list(ErrorKnn.values()).index(minErrorKnn)]
    argMinKnn += argmin
argMinKnn /= Nb_boucle
print("Error",ErrorKnn)
print("argminlast K = ",argmin)
print("argminmoy K = ",argMinKnn)
argMinKnn = round(argMinKnn)
print("Param retenu K = ",argMinKnn)
print("minErrorKnn", minErrorKnn)


"""
X_train_train, X_train_test, y_train_train,y_train_test = train_test_split(X, y, test_size=50, random_state=666, shuffle=True)
for i in range(len(listKnn)):
        listKnn[i].fit(X_train_train, y_train_train)
        ErrorKnn[listKnn[i].C, listKnn[i].gamma] = 1 - listKnn[i].score(X_train_test, y_train_test)
minErrorKnn = min([ErrorKnn[i] for i in ErrorKnn.keys()])
argMinKnn = list(ErrorKnn.keys())[list(ErrorKnn.values()).index(minErrorKnn)]
print("Error",ErrorKnn)
print("argmin",argMinKnn)
print("minErrorKnn", minErrorKnn)

"""


_times.append(time.time())

print("temps : ", _times[-1] - _times[-2])

Nombre de classifieurs :  9
Error {1: 0.01610092525394411, 2: 0.01610092525394411, 3: 0.01647497922042442, 4: 0.01614524665950864, 5: 0.01880597823154606, 6: 0.018727381922712626, 7: 0.020057190432927367, 8: 0.020341367530945287, 9: 0.0217101284465513}
argminlast K =  1
argminmoy K =  2.6
Param retenu K =  3
minErrorKnn 0.01610092525394411
temps :  42.46193218231201


In [6]:
int(1.9)

1

In [7]:
_times = []
_times.append(time.time())

MatrixSigma =  numpy.array(pairwise_distances(X_train))
SigmaMin = (MatrixSigma + numpy.identity(MatrixSigma.shape[0])*1e10).min()
SigmaParam = numpy.array([SigmaMin*(2**i) for i in range(6)])
listGamma = 1.0/(2*(SigmaParam**2))
#listC = [10**n for n in range(-5,6)]
listC = list(range(60,100,5))
listSVC = []
ErrorSVC = {(c,g):0 for c in listC for g in listGamma}

for g in listGamma:
    for c in listC:
        listSVC.append(SVC(kernel='rbf', C=c, gamma=g))

print("Nombre de classifieurs : ",len(listSVC))


kf = KFold(n_splits=5, shuffle=True)
for train_index, test_index in kf.split(X_train):
    X_train_train = X_train[train_index]
    X_train_test = X_train[test_index]
    y_train_train = y_train[train_index]
    y_train_test = y_train[test_index]
    for i in range(len(listSVC)):
        listSVC[i].fit(X_train_train, y_train_train)
        ErrorSVC[listSVC[i].C, listSVC[i].gamma] += 1 - listSVC[i].score(X_train_test, y_train_test)
ErrorSVC = {k: v / kf.get_n_splits(X_train) for k, v in ErrorSVC.items()}
minErrorSVC = min([ErrorSVC[i] for i in ErrorSVC.keys()])
argMinSVC = list(ErrorSVC.keys())[list(ErrorSVC.values()).index(minErrorSVC)]
#print("Error",ErrorSVC)
print("argmin C, g = ",argMinSVC)
print("minErrorSvc", minErrorSVC)


"""
X_train_train, X_train_test, y_train_train,y_train_test = train_test_split(X, y, test_size=50, random_state=666, shuffle=True)
for i in range(len(listSVC)):
        listSVC[i].fit(X_train_train, y_train_train)
        ErrorSVC[listSVC[i].C, listSVC[i].gamma] = 1 - listSVC[i].score(X_train_test, y_train_test)
minErrorSVC = min([ErrorSVC[i] for i in ErrorSVC.keys()])
argMinSVC = list(ErrorSVC.keys())[list(ErrorSVC.values()).index(minErrorSVC)]
print("Error",ErrorSVC)
print("argmin",argMinSVC)
print("minErrorSVC", minErrorSVC)
"""



_times.append(time.time())

print("temps : ", _times[-1] - _times[-2])

Nombre de classifieurs :  48
argmin C, g =  (60, 0.6973101806565765)
minErrorSvc 0.0050000000000000044
temps :  240.44221758842468


In [8]:
listGamma

array([1.78511406e+02, 4.46278516e+01, 1.11569629e+01, 2.78924072e+00,
       6.97310181e-01, 1.74327545e-01])

In [9]:
_times = []
_times.append(time.time())

Nfeat = len(X[0])
Nsortie = len(set(y_train))
listHidLay1 = [Nfeat,2*Nfeat,3*Nfeat]
listHidLay2 = [Nsortie,2*Nsortie,3*Nsortie]
listHidLay = [tuple(listHidLay1[i:i+1]) for i in range(len(listHidLay1))]+[(x,y) for x in listHidLay1 for y in listHidLay2] 
print(listHidLay1)
print(listHidLay2)
print(listHidLay)
print("Nombre de classifieurs : ",len(listHidLay))
listMLP = list(map(lambda HidLay:MLPClassifier(hidden_layer_sizes=HidLay) , listHidLay))
ErrorMLP = {listMLP[i].hidden_layer_sizes:0 for i in range(len(listHidLay))}

kf = KFold(n_splits=3, shuffle=True)
for train_index, test_index in kf.split(X_train):
    X_train_train = X_train[train_index]
    X_train_test = X_train[test_index]
    y_train_train = y_train[train_index]
    y_train_test = y_train[test_index]
    for i in range(len(listMLP)):
        listMLP[i].fit(X_train_train, y_train_train)
        ErrorMLP[listMLP[i].hidden_layer_sizes] += 1 - listMLP[i].score(X_train_test, y_train_test)
#ErrorMLP = {k: v / kf.get_n_splits(X_train) for k, v in ErrorMLP.items()}
minErrorMLP = min([ErrorMLP[i] for i in ErrorMLP.keys()])
argMinMLP = list(ErrorMLP.keys())[list(ErrorMLP.values()).index(minErrorMLP)]
#print("Error",ErrorMLP)
print("argmin Couche cachée 1, couche cachée 2 = ",argMinMLP)
print("minErrorMLP", minErrorMLP)


"""
X_train_train, X_train_test, y_train_train,y_train_test = train_test_split(X, y, test_size=50, shuffle=True)
for i in range(len(listMLP)):
        listMLP[i].fit(X_train_train, y_train_train)
        ErrorMLP[listMLP[i].hidden_layer_sizes] = 1 - listMLP[i].score(X_train_test, y_train_test)
minErrorMLP = min([ErrorMLP[i] for i in ErrorMLP.keys()])
argMinMLP = list(ErrorMLP.keys())[list(ErrorMLP.values()).index(minErrorMLP)]
print("Error",ErrorMLP)
print("argmin",argMinMLP)
print("minErrorMLP", minErrorMLP)
"""


_times.append(time.time())

print("temps : ", _times[-1] - _times[-2])

[16, 32, 48]
[10, 20, 30]
[(16,), (32,), (48,), (16, 10), (16, 20), (16, 30), (32, 10), (32, 20), (32, 30), (48, 10), (48, 20), (48, 30)]
Nombre de classifieurs :  12
argmin Couche cachée 1, couche cachée 2 =  (48, 30)
minErrorMLP 0.04919736340847092
temps :  79.78856825828552


In [10]:
ClassMLP = MLPClassifier(hidden_layer_sizes=argMinMLP)
ClassSVC = SVC(kernel='rbf', C=argMinSVC[0], gamma=argMinSVC[1])
ClassKnn = KNeighborsClassifier(n_neighbors=argMinKnn)
kf = KFold(n_splits=5, shuffle=True)
ErrorKnn, ErrorSVC,ErrorMLP = 0, 0, 0
for train_index, test_index in kf.split(X_test):
    X_test_train = X_test[train_index]
    X_test_test = X_test[test_index]
    y_test_train = y_test[train_index]
    y_test_test = y_test[test_index]
    ClassMLP.fit(X_test_train, y_test_train)
    ClassSVC.fit(X_test_train, y_test_train)
    ClassKnn.fit(X_test_train, y_test_train)
    ErrorMLP += 1 - ClassMLP.score(X_test_test, y_test_test)
    ErrorSVC += 1 - ClassSVC.score(X_test_test, y_test_test)
    ErrorKnn += 1 - ClassKnn.score(X_test_test, y_test_test)

print("ErrorMLP : ",ErrorMLP/kf.get_n_splits(X_test))
print("ErrorSVC : ",ErrorSVC/kf.get_n_splits(X_test))
print("ErrorKnn : ",ErrorKnn/kf.get_n_splits(X_test))

ErrorMLP :  0.013184888352981972
ErrorSVC :  0.005507789602075164
ErrorKnn :  0.008845016924231498
